In [ ]:
!pip install deap update_checker tqdm stopit tpot

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 MB 6.1 MB/s eta 0:00:00
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=7065ef99bcddd8daf8c97352bf61eac436c3f4dd6299b7cbec8d9ade36110a48
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.

In [ ]:
import pandas as pd
import numpy as np
from tpot import TPOTClassifier


df_test=pd.read_csv("TestSet_klassification.csv")
df_train=pd.read_csv("TrainSet_klassification.csv")

X_train = df_train.drop(["income"], axis=1).values
y_train = df_train["income"].values

X_test = df_test.drop(["income"], axis=1).values
y_test = df_test["income"].values

In [ ]:
# initializing tpot with parameters
#max_time_mins=None (per default) - here max time 4h
tpot = TPOTClassifier(generations=5, population_size=50,
                     cv=5, verbosity=2, n_jobs=-1, max_time_mins=240,
                     periodic_checkpoint_folder='/content/results')

In [ ]:
#starting the training
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8726204798373307

Generation 2 - Current best internal CV score: 0.8736132321825232

Best pipeline: MLPClassifier(input_matrix, alpha=0.1, learning_rate_init=0.01)


TPOTClassifier(cv=2, generations=2, max_time_mins=4, n_jobs=-1,
               periodic_checkpoint_folder='/content/results', population_size=5,
               verbosity=2)

In [ ]:
import sklearn.metrics as skm
from sklearn.metrics import roc_auc_score

#prediction
y_pred = tpot.predict(X_test)

# prediction for the AUC-ROC-Score by only using the positive classes
y_pred_proba = tpot.predict_proba(X_test)[:, 1]


print("RESULTS OF BEST MODEL:\n")

print(f"F1-Score:                   {skm.f1_score(y_test, y_pred)}")
print(f"AUC-ROC Score:              {roc_auc_score(y_test, y_pred_proba)}")
print(f"Accuracy:                   {skm.accuracy_score(y_test, y_pred)}")
print(f"Precision:                  {skm.precision_score(y_test, y_pred)}")
print(f"Recall:                     {skm.recall_score(y_test, y_pred)}")

RESULTS OF BEST MODEL:

F1-Score:                   0.5868452146514376
AUC-ROC Score:              0.8578269902032121
Accuracy:                   0.8066359447004608
Precision:                  0.5159279778393352
Recall:                     0.680365296803653


In [ ]:
from google.colab import files
# output of values to dictionary
metrics_dict = {
    'Metric': [
        'F1-Score',
        'AUC-ROC Score',
        'Accuracy',
        'Precision',
        'Recall',
    ],
    'Value': [
        skm.f1_score(y_test, y_pred),
        roc_auc_score(y_test, y_pred_proba),
        skm.accuracy_score(y_test, y_pred),
        skm.precision_score(y_test, y_pred),
        skm.recall_score(y_test, y_pred)
    ]
}

#to df
metrics_df = pd.DataFrame(metrics_dict)


print(metrics_df)

metrics_df.to_csv('tpot_classification_preprosessing_metrics.csv', index=False)
files.download('tpot_classification_preprosessing_metrics.csv')

          Metric     Value
0       F1-Score  0.586845
1  AUC-ROC Score  0.857827
2       Accuracy  0.806636
3      Precision  0.515928
4         Recall  0.680365


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
tpot.export('tpot_classification_preprocessing_model')